# Dependencies

In [37]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [38]:
!pip install -q kaggle

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Dropout, LayerNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from tensorflow import keras
%load_ext tensorboard


import os

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Data Loading

The code defines a function `create_kaggle_dir()` to create a directory named ".kaggle" in the root directory "/root/" if it does not exist. 

The next function, `retrieve_kaggle_json()`, retrieves the kaggle.json file from a specified source directory and places it in the ".kaggle" directory. The file is used for authentication when using the Kaggle API. The function sets appropriate file permissions using chmod.

The next two functions, `download_facemask_dataset()` and `unzip_facemask_dataset()`, download and unzip a face mask detection dataset respectively if it does not already exist in the "/content/" directory.

The final function, `dataset_pipeline()`, calls all the above functions in the following order - `create_kaggle_dir()`, `retrieve_kaggle_json()`, `download_facemask_dataset()`, and `unzip_facemask_dataset()`.

The last line of the code executes the `dataset_pipeline()` function. This code can be useful for setting up the required dataset for a machine learning project related to face mask detection.

In [40]:
def create_kaggle_dir(kaggle_dir="/root/.kaggle/"):
    if not os.path.exists(kaggle_dir):
        !mkdir $kaggle_dir

def retrieve_kaggle_json(source_dir="/content/drive/MyDrive/", kaggle_dir="/root/.kaggle/", file_name="kaggle.json"):
    source_file = os.path.join(source_dir, file_name)
    target_file = os.path.join(kaggle_dir, file_name)

    if not os.path.exists(target_file):
        !cp $source_file $kaggle_dir
        !chmod 600 $target_file

def download_facemask_dataset():
    if not os.path.exists("/content/face-mask-detection.zip"):
        !kaggle datasets download -d dhruvmak/face-mask-detection

def unzip_facemask_dataset():
    if not os.path.exists("/content/face-mask-detection"):
        !unzip face-mask-detection.zip -d face-mask-detection > /dev/null 2>&1

def dataset_pipeline():
    create_kaggle_dir()
    retrieve_kaggle_json()
    download_facemask_dataset()
    unzip_facemask_dataset()

dataset_pipeline()

In [54]:
IMG_SIZE = 224

datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0, 
                                                          shear_range=0.2, 
                                                          zoom_range=0.2, 
                                                          rotation_range=45, 
                                                          horizontal_flip=True, 
                                                          validation_split=0.2)

The code uses the Keras `ImageDataGenerator` class to create two data generators named `train` and `test`. 

The `flow_from_directory()` method of the `ImageDataGenerator` class is used to load the training and validation datasets from the specified directory '/content/face-mask-detection/dataset/'. The images in the directory are resized to the specified `IMG_SIZE` and augmented by the `datagen` object. 

Both the training and validation datasets are loaded as batches of 16 images at a time, with the `class_mode` argument set to 'binary' as there are only two classes (with mask and without mask) in the dataset. The `interpolation` argument is set to 'lanczos' for better image quality. 

The `subset` argument is set to 'training' for the `train` data generator and 'validation' for the `test` data generator to specify the type of dataset to be loaded. Finally, the `shuffle` argument is set to `True` to shuffle the order of the images during training and testing.

In [55]:
train = datagen.flow_from_directory('/content/face-mask-detection/dataset/', target_size=(IMG_SIZE,IMG_SIZE), batch_size=16, 
                                    class_mode='binary', interpolation='lanczos', shuffle=True, subset='training')

test = datagen.flow_from_directory('/content/face-mask-detection/dataset/', target_size=(IMG_SIZE,IMG_SIZE), batch_size=16, 
                                    class_mode='binary', interpolation='lanczos', shuffle=True, subset='validation')

Found 352 images belonging to 2 classes.
Found 88 images belonging to 2 classes.


# Model Creation

The following code defines a deep learning model using the Keras `Sequential` API. 

The model architecture consists of several layers, including a pre-trained `MobileNet` layer, followed by a `Conv2D` layer, a `Flatten` layer, and a `Dense` layer. 

The input shape of the model is specified using the `Input` layer. The `IMG_SIZE` is used to set the height and width of the input image, and `3` specifies the number of color channels.

The model is compiled using the `compile()` method, which specifies the loss function, optimizer, and evaluation metric. The `binary_crossentropy` loss function is used since the dataset has only two classes. The `rmsprop` optimizer is used to minimize the loss function during training, and `accuracy` is used as the evaluation metric.

Finally, the `summary()` method is used to display the model summary, which shows the layer-wise structure of the model and the number of parameters in each layer.

In [50]:
model_path = "/content/drive/MyDrive/Lab-Proj/models/mobilenet-facemask2.h5"

if os.path.exists(model_path):
    print("Loading existing model . . .")
    model = keras.models.load_model(model_path)

else:
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input((IMG_SIZE,IMG_SIZE,3)),
        tf.keras.applications.MobileNet(include_top=False, weights='imagenet'),
        tf.keras.layers.Conv2D(3, 3, padding='same', activation='sigmoid'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.fit(train, epochs=5, validation_data=test)

    model.save(model_path)

Epoch 1/5
22/22 [==============================] - 27s 712ms/step - loss: 0.2794 - accuracy: 0.8778 - val_loss: 0.3178 - val_accuracy: 0.8864
Epoch 2/5
22/22 [==============================] - 15s 687ms/step - loss: 0.1057 - accuracy: 0.9716 - val_loss: 0.1822 - val_accuracy: 0.9886
Epoch 3/5
22/22 [==============================] - 17s 809ms/step - loss: 0.0742 - accuracy: 0.9830 - val_loss: 0.1075 - val_accuracy: 0.9773
Epoch 4/5
22/22 [==============================] - 16s 746ms/step - loss: 0.0508 - accuracy: 0.9830 - val_loss: 0.0707 - val_accuracy: 0.9886
Epoch 5/5
22/22 [==============================] - 15s 680ms/step - loss: 0.0250 - accuracy: 0.9972 - val_loss: 0.0540 - val_accuracy: 0.9886


In [51]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, None, None, 1024)  3228864  
 nal)                                                            
                                                                 
 conv2d_15 (Conv2D)          (None, 7, 7, 3)           27651     
                                                                 
 flatten_17 (Flatten)        (None, 147)               0         
                                                                 
 dense_21 (Dense)            (None, 1)                 148       
                                                                 
Total params: 3,256,663
Trainable params: 3,234,775
Non-trainable params: 21,888
_________________________________________________________________


In [52]:
model.evaluate(test)

6/6 [==============================] - 3s 435ms/step - loss: 0.0340 - accuracy: 1.0000


[0.034043919295072556, 1.0]

## Predict Custom Images

The code defines a function named `predict` that takes a file path as input.

The function loads an image from the specified file path using the `load_img` method of `keras.preprocessing.image`. The image is then resized to the specified `IMG_SIZE` and converted to a NumPy array using the `img_to_array` method. 

The pixel values of the image are normalized by dividing with 255.0. The model is then used to predict whether the image contains a person wearing a mask or not, by passing the processed image as input to the `predict` method of the `model` object. 

The predicted result is stored in the `pred` variable as a probability value. The image is then plotted using `matplotlib.pyplot.imshow` and a title is added to the plot using `plt.title`. The title displays the predicted class and the corresponding probability score in percentage.

If the predicted probability value is greater than 0.5, the image is classified as 'Without Mask' and if it is less than or equal to 0.5, the image is classified as 'With Mask'.

In [ ]:
def predict(path) :
    img = tf.keras.preprocessing.image.load_img(path, target_size=(IMG_SIZE,IMG_SIZE), interpolation='lanczos')
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = img / 255.0
    pred = model.predict(np.array([img]))
    plt.imshow(img)
    if pred[0][0] > 0.5 : 
        plt.title(f'Without Mask : {pred[0][0]*100 : 0.2f}%')
    else : 
        plt.title(f'With Mask : {(1-pred[0][0])*100 : 0.2f}%')

In [ ]:
path = "/content/drive/MyDrive/Lab-Proj/testing/no-mask-3.jpeg"
predict(path)

# Models Variants

In [57]:
IMG_SIZE = 224

datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

The code uses the Keras `ImageDataGenerator` class to create two data generators named `train` and `test`. 

The `flow_from_directory()` method of the `ImageDataGenerator` class is used to load the training and validation datasets from the specified directory '/content/face-mask-detection/dataset/'. The images in the directory are resized to the specified `IMG_SIZE` and augmented by the `datagen` object. 

Both the training and validation datasets are loaded as batches of 16 images at a time, with the `class_mode` argument set to 'binary' as there are only two classes (with mask and without mask) in the dataset. The `interpolation` argument is set to 'lanczos' for better image quality. 

The `subset` argument is set to 'training' for the `train` data generator and 'validation' for the `test` data generator to specify the type of dataset to be loaded. Finally, the `shuffle` argument is set to `True` to shuffle the order of the images during training and testing.

In [67]:
train = datagen.flow_from_directory('/content/face-mask-detection/dataset/', target_size=(IMG_SIZE,IMG_SIZE), batch_size=16, 
                                    class_mode='binary', interpolation='lanczos', shuffle=True, subset='training')

test = datagen.flow_from_directory('/content/face-mask-detection/dataset/', target_size=(IMG_SIZE,IMG_SIZE), batch_size=16, 
                                    class_mode='binary', interpolation='lanczos', shuffle=False, subset='validation')

Found 352 images belonging to 2 classes.
Found 88 images belonging to 2 classes.


The following code defines a deep learning model using the Keras `Sequential` API, defining an heavier and more performing variant of the previous `MobileNet` model. 

The model architecture consists of several layers, including a pre-trained `ResNet50` layer, followed by a `Conv2D` layer, a `Flatten` layer, and a `Dense` layer. 

The input shape of the model is specified using the `Input` layer. The `IMG_SIZE` is used to set the height and width of the input image, and `3` specifies the number of color channels.

The model is compiled using the `compile()` method, which specifies the loss function, optimizer, and evaluation metric. The `binary_crossentropy` loss function is used since the dataset has only two classes. The `rmsprop` optimizer is used to minimize the loss function during training, and `accuracy` is used as the evaluation metric.

Finally, the `summary()` method is used to display the model summary, which shows the layer-wise structure of the model and the number of parameters in each layer.

In [68]:
model_path = "/content/drive/MyDrive/Lab-Proj/models/resnet-facemask.h5"

if os.path.exists(model_path):
    print("Loading existing model . . .")
    model = keras.models.load_model(model_path)

else:
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input((IMG_SIZE,IMG_SIZE,3)),
        tf.keras.applications.ResNet50(include_top=False, weights='imagenet'),
        tf.keras.layers.Conv2D(3, 3, padding='same', activation='sigmoid'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    model.fit(train, epochs=50, validation_data=test)
    model.save(model_path)

Epoch 1/50
22/22 [==============================] - 43s 593ms/step - loss: 0.3193 - accuracy: 0.8750 - val_loss: 0.9493 - val_accuracy: 0.5000
Epoch 2/50
22/22 [==============================] - 11s 479ms/step - loss: 0.1066 - accuracy: 0.9716 - val_loss: 1.5043 - val_accuracy: 0.5000
Epoch 3/50
22/22 [==============================] - 12s 519ms/step - loss: 0.0332 - accuracy: 0.9972 - val_loss: 1.3728 - val_accuracy: 0.5000
Epoch 4/50
22/22 [==============================] - 12s 526ms/step - loss: 0.0747 - accuracy: 0.9688 - val_loss: 0.7381 - val_accuracy: 0.5795
Epoch 5/50
22/22 [==============================] - 12s 532ms/step - loss: 0.0642 - accuracy: 0.9858 - val_loss: 0.1560 - val_accuracy: 0.9432
Epoch 6/50
22/22 [==============================] - 12s 515ms/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.0773 - val_accuracy: 0.9773
Epoch 7/50
22/22 [==============================] - 11s 524ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0486 - val_accuracy: 0.9886

In [69]:
model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 conv2d_22 (Conv2D)          (None, 7, 7, 3)           55299     
                                                                 
 flatten_24 (Flatten)        (None, 147)               0         
                                                                 
 dense_28 (Dense)            (None, 1)                 148       
                                                                 
Total params: 23,643,159
Trainable params: 23,590,039
Non-trainable params: 53,120
_________________________________________________________________


In [70]:
model.evaluate(test)

6/6 [==============================] - 2s 258ms/step - loss: 0.0478 - accuracy: 0.9886


[0.04780176654458046, 0.9886363744735718]